In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import subprocess
import os

pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Understanding

* RKI, webscrape (webscraping) https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html
* John Hopkins (GITHUB) https://github.com/CSSEGISandData/COVID-19.git
* REST API services to retreive data https://npgeo-corona-npgeo-de.hub.arcgis.com/

In [2]:
data_path = '../data/raw/time_series_covid19_confirmed_global.csv'
pd_raw = pd.read_csv(data_path)

In [3]:
pd_raw

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/14/22,5/15/22,5/16/22,5/17/22,5/18/22,5/19/22,5/20/22,5/21/22,5/22/22,5/23/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,179242,179267,179321,179328,179477,179597,179624,179674,179716,179716
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,275574,275615,275621,275688,275732,275732,275732,275838,275864,275881
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,265816,265818,265823,265828,265834,265841,265847,265851,265854,265855
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,42156,42156,42156,42156,42572,42572,42572,42572,42572,42572
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,99287,99287,99287,99287,99287,99287,99287,99287,99287,99287
5,NaN,Antarctica,-71.949900,23.347000,0,0,0,0,0,0,...,11,11,11,11,11,11,11,11,11,11
6,NaN,Antigua and Barbuda,17.060800,-61.796400,0,0,0,0,0,0,...,7721,7721,7795,7795,7855,7910,7910,7942,7942,7982
7,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,9101319,9135308,9135308,9135308,9135308,9135308,9135308,9135308,9178795,9178795
8,NaN,Armenia,40.069100,45.038200,0,0,0,0,0,0,...,422900,422900,422917,422917,422917,422917,422917,422917,422917,422939
9,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,118728,119548,120638,121713,122718,123655,124477,125220,125828,126633


## Web Scraping

In [4]:
page = requests.get('https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html')

In [5]:
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
soup.get_text()

'\n\n\n\n\nRKI  -  Coronavirus SARS-CoV-2 - COVID-19: Fallzahlen in Deutschland und weltweit\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation und Service\nSpringe direkt zu:\n\nInhalt\nHauptmenu\nSuche\n\n\n\n\n\n\n\nServicemenü\nKon\xadtaktIn\xadhaltHil\xadfeIm\xadpres\xads\xadumDa\xadten\xadschut\xadz\xader\xadklä\xadrungRSSEnglish\n\n\n\n\nErklärung zur Barrierefreiheit\n\n\nGebärdensprache\n\n\nLeichte Sprache\n\n\n\n\nSuche\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nKontakt\nNavigation\n\n\n\nZielgruppeneinstiege\n\n\n\n\nInfektionskrankheiten A-Z\n\n\n\nA\nB\nC\nD\nE\nF\nG\nH\nI\nJ\nK\nL\nM\nN\nO\nP\nQ\nR\nS\nT\nU\nV\nW\nX\nY\nZ\nalle\n\n\n\n\n\n\n\n\nGesundheit A-Z\n\n\n\nA\nB\nC\nD\nE\nF\nG\nH\nI\nJ\nK\nL\nM\nN\nO\nP\nQ\nR\nS\nT\nU\nV\nW\nX\nY\nZ\nalle\n\n\n\n\n\n\n\nNavigation\n\nIn\xadsti\xadtut\nGe\xadsund\xadheits\xadmo\xadni\xadto\xadring\nIn\xadfek\xadti\xadons\xadschutz\nForschung\nKom\xadmis\xadsio\xadnen\nSer\xadvice\n\n\n\n\n\n\nZiel

In [7]:
html_table = soup.find('table')

In [8]:
all_rows = html_table.find_all('tr')

In [9]:
final_data_list = []
for pos, rows in enumerate(all_rows):
    col_list = [each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)

In [10]:
pd_daily_status = pd.DataFrame(final_data_list).dropna().rename(columns={0:'state',
                                                       1:'cases',
                                                       2:'changes',
                                                       3:'cases_in_7days',
                                                       4:'7_day_incidence',
                                                       5:'deaths'})
pd_daily_status.head()

,state,cases,changes,cases_in_7days,7_day_incidence,deaths
2,Baden-Württem­berg,3.647.814,7.705,30.435,"274,1",16.055
3,Bayern,4.863.054,11.384,43.854,"333,7",23.979
4,Berlin,1.041.177,2.183,9.167,"250,2",4.604
5,Branden­burg,786.494,883,4.601,"181,8",5.654
6,Bremen,198.904,775,3.166,"465,5",773


## REST API calls

In [11]:
data = requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronafälle_in_den_Bundesländern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [12]:
json_object = json.loads(data.content)

In [13]:
type(json_object)

dict

In [14]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'geometryType', 'spatialReference', 'fields', 'features'])

In [15]:
full_list = []
for pos,each_dict in enumerate(json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [16]:
pd_full_list = pd.DataFrame(full_list)
pd_full_list.head()

,OBJECTID_1,LAN_ew_AGS,LAN_ew_GEN,LAN_ew_BEZ,LAN_ew_EWZ,OBJECTID,Fallzahl,Aktualisierung,AGS_TXT,GlobalID,faelle_100000_EW,Shape__Area,Shape__Length,Death,cases7_bl_per_100k,cases7_bl,death7_bl,cases7_bl_per_100k_txt,AdmUnitId
0,1,01,Schleswig-Holstein,Land,2910875,15,724402,1653343200000,01,fc5ba936-c95c-432c-8a33-9eb2f30b660f,24886.056598,4.573731e+10,2.881496e+06,2516,462.644394,13467,2,"462,6",1
1,2,02,Hamburg,Freie und Hansestadt,1852478,6,577907,1653343200000,02,0f3e860c-5181-4d3f-a421-1d51f50315ea,31196.429863,2.089396e+09,4.188002e+05,2643,332.905438,6167,1,"332,9",2
2,3,03,Niedersachsen,Land,8003421,9,2335583,1653343200000,03,3fd77024-c29b-4843-9be8-682ad48e60c9,29182.308415,1.299836e+11,4.008988e+06,9232,412.736004,33033,7,"412,7",3
3,4,04,Bremen,Freie Hansestadt,680130,5,198904,1653343200000,04,4132268b-54de-4327-ac1e-760e915112f1,29244.997280,1.119157e+09,3.357177e+05,773,465.499243,3166,2,"465,5",4
4,5,05,Nordrhein-Westfalen,Land,17925570,10,5230619,1653343200000,05,561d658f-3ee5-46e3-bc95-3528c6558ab9,29179.652307,8.782936e+10,2.648673e+06,25297,319.855938,57336,6,"319,9",5


## API Access via REST service, e.g. USA data

In [20]:
# US for full list

response = requests.get('https://api.covidtracking.com/v2/us/daily.json')
print(response)

<Response [200]>


In [24]:
US_dict=json.loads(response.content) # imports string
US_dict.keys()

dict_keys(['links', 'meta', 'data'])

In [ ]:
full_list = []
for pos,each_dict in enumerate(json_object['features'][:]):
    full_list.append(each_dict['attributes'])